In [2]:
print('hi')

hi


In [3]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

/Users/aiant/AI_project/ai_venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/aiant/AI_project/ai_venv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [5]:
from transformers import BertForNextSentencePrediction
nsp_model = BertForNextSentencePrediction.from_pretrained('bert-base-uncased')
nsp_model.eval()

/Users/aiant/AI_project/ai_venv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForNextSentencePrediction: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForNextSentencePrediction from the checkpoin

BertForNextSentencePrediction(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [6]:
import torch
def show_continuity(text1, text2):
    tokenized = tokenizer(text1, text2, return_tensors='pt')
    y = nsp_model(**tokenized)
    print("Result: ", y)
    pred = torch.softmax(y.logits, dim=1)
    print(str(pred[0][0].item()*100) + "%의 확률로 연속되어 있습니다.")

In [7]:
text = "[CLS] I played baseball with my friends at school yesterday [SEP]"
words = tokenizer.tokenize(text)
print(words)

['[CLS]', 'i', 'played', 'baseball', 'with', 'my', 'friends', 'at', 'school', 'yesterday', '[SEP]']


In [8]:
msk_idx = 3
words[msk_idx] = "[MASK]"
print(words)

['[CLS]', 'i', 'played', '[MASK]', 'with', 'my', 'friends', 'at', 'school', 'yesterday', '[SEP]']


In [9]:
word_ids = tokenizer.convert_tokens_to_ids(words)
word_tensor = torch.tensor([word_ids])
print(word_tensor)

tensor([[ 101, 1045, 2209,  103, 2007, 2026, 2814, 2012, 2082, 7483,  102]])


In [10]:
from transformers import BertForMaskedLM

# 학습 완료 모델을 불러옴
msk_model = BertForMaskedLM.from_pretrained('bert-base-uncased') 
msk_model.eval() # 학습을 수행하지 않으므로 평가 모드로 함


/Users/aiant/AI_project/ai_venv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [11]:
x = word_tensor
y = msk_model(x)
result = y[0]
print(result.size())
# torch.Size([1, 11, 30522]) : 1개의 문장에 11개의 단어가 있고, 30522개의 단어 집합에서 각 단어의 점수를 나타냄

torch.Size([1, 11, 30522])


In [12]:
_, max_ids = torch.topk(result[0][msk_idx], k=5)
result_words = tokenizer.convert_ids_to_tokens(max_ids.tolist())
print(result_words)


['basketball', 'football', 'soccer', 'baseball', 'tennis']


In [13]:
from transformers import BertForNextSentencePrediction

nsp_model = BertForNextSentencePrediction.from_pretrained('bert-base-uncased')
nsp_model.eval()

/Users/aiant/AI_project/ai_venv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForNextSentencePrediction: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForNextSentencePrediction from the checkpoin

BertForNextSentencePrediction(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [15]:
# 문장의 연속성을 판단하는 함수
def show_continuity(text1, text2):
    tokenized = tokenizer(text1, text2, return_tensors='pt') # return_tensors='pt' 데이터는 파이토치의 텐서로 변환
    print("Tokenized: ", tokenized)
    y = nsp_model(**tokenized)
    print("Result: ", y)
    pred = torch.softmax(y.logits, dim=1)
    print(str(pred[0][0].item()*100) + "%의 확률로 연속되어 있습니다.")
    
text1 = "What is baseball?"
text2 = "It is a game of hitting the vall with a bat."
show_continuity(text1, text2)

Tokenized:  {'input_ids': tensor([[  101,  2054,  2003,  3598,  1029,   102,  2009,  2003,  1037,  2208,
          1997,  7294,  1996, 11748,  2140,  2007,  1037,  7151,  1012,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
Result:  NextSentencePredictorOutput(loss=None, logits=tensor([[ 6.1284, -5.8279]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
99.99935626983643%의 확률로 연속되어 있습니다.


In [17]:
text1 = "What is baseball?"
# text2 = "I played baseball with my friends at school yesterday." 
text2 = "This food is made with flour and milk."
show_continuity(text1, text2)

Tokenized:  {'input_ids': tensor([[  101,  2054,  2003,  3598,  1029,   102,  2023,  2833,  2003,  2081,
          2007, 13724,  1998,  6501,  1012,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
Result:  NextSentencePredictorOutput(loss=None, logits=tensor([[-4.1025,  7.1723]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
0.0012688289643847384%의 확률로 연속되어 있습니다.
